<a href="https://colab.research.google.com/github/nutworker/qM-AI-L/blob/L_test/email-subject/model-tuning/FB_Bart_Model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
repository_url = 'https://github.com/nutworker/qM-AI-L'
!git clone {repository_url}

Cloning into 'qM-AI-L'...
remote: Enumerating objects: 243, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 243 (delta 63), reused 27 (delta 27), pack-reused 162
Receiving objects: 100% (243/243), 7.68 MiB | 4.82 MiB/s, done.
Resolving deltas: 100% (123/123), done.


# **Setting Up the Environment**

In [ ]:
!pip install datasets

In [8]:

# Importing Libraries

import os
import re
import pandas as pd # Data Handling
import tensorflow as tf
import numpy as np
import time
# import seaborn as sns

from datasets import load_metric
from datasets import Dataset

AttributeError: module 'pyarrow.lib' has no attribute 'ListViewType'

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
!pip install transformers
!pip install langchain_community
!pip install langchain
# !pip install unstructured
!pip install evaluate
!pip install rouge-score # Installing rouge-score library (https://pypi.org/project/rouge-score/)

# !pip install accelerate -U

!pip install transformers[torch]
!pip install sacrebleu

!pip install datasets nltk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Cloning into 'AESLC'...
remote: Enumerating objects: 17469, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 17469 (delta 1), reused 0 (delta 0), pack-reused 17461
Receiving objects: 100% (17469/17469), 7.36 MiB | 3.63 MiB/s, done.
Resolving deltas: 100% (48/48), done.
Updating files: 100% (18304/18304), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.9/378.9 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 25.1 MB/s eta 0:00:00
   ━━━━

In [15]:
!pip install \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
#     loralib==0.1.1 \
#     peft==0.3.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: evaluate
    Found existing installation: evaluate 0.4.2
    Uninstalling evaluate-0.4.2:
      Successfully uninstalled evaluate-0.4.2


In [12]:
!pip install --upgrade pyarrow

In [1]:
# Transformers
from transformers import BartTokenizer, BartForConditionalGeneration      # BERT Tokenizer and architecture
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments         # These will help us to fine-tune our model
from transformers import pipeline                                         # Pipeline
from transformers import DataCollatorForSeq2Seq                           # DataCollator to batch the data
import torch

In [18]:
!pip uninstall -y pyarrow
!pip install pyarrow

Found existing installation: pyarrow 17.0.0
Uninstalling pyarrow-17.0.0:
  Successfully uninstalled pyarrow-17.0.0
  Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (39.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import evaluate

In [3]:
dataset_df = pd.read_csv('/content/qM-AI-L/email-subject/model-tuning/train_emails.csv')
dataset_df

,source,body,subject,ann0,ann1,ann2,body_wcount,subj_wcount,cleaned_emails
0,allen-p_inbox_20,"Greg/Phillip, Attached is the Grande Communic...",Service Agreement,NaN,NaN,NaN,65,2,gregphillip attached is the grande communicati...
1,allen-p_inbox_28,Phillip & Keith Attached is the first draw re...,Bishops Corner,NaN,NaN,NaN,145,2,phillip keith attached is the first draw reque...
2,allen-p_inbox_63,Your Internet Banking accounts are now setup a...,Internet Banking,NaN,NaN,NaN,250,2,your internet banking accounts are now setup a...
3,allen-p_inbox_64,To our IBS Customers that are still hanging in...,Internet Banking,NaN,NaN,NaN,458,2,to our ibs customers that are still hanging in...
4,allen-p_inbox_65,Phillip Good Morning!\nI hope you had a wonder...,SMEs for expert stories,NaN,NaN,NaN,68,4,phillip good morning i hope you had a wonderfu...
...,...,...,...,...,...,...,...,...,...
14431,zufferli-j_inbox_43,This email is acknowledgement from the Power P...,Power Pool,NaN,NaN,NaN,227,2,this email is acknowledgement from the power p...
14432,zufferli-j_inbox_44,This email is acknowledgement from the Power P...,Power Pool of Alberta,NaN,NaN,NaN,277,4,this email is acknowledgement from the power p...
14433,zufferli-j_inbox_46,"John, Further to the voice message that I lef...",Enron Security,NaN,NaN,NaN,148,2,john further to the voice message that i left ...
14434,zufferli-j_inbox_8,Make sure that all curves are downloaded by th...,Simulation Curves,NaN,NaN,NaN,66,2,make sure that all curves are downloaded by th...


In [30]:
subset_df = dataset_df.sample(frac=0.4, random_state=42)
subset_df.shape


(5774, 9)

In [31]:
Train_df, Test_df = train_test_split(subset_df, test_size=0.2, random_state=42)
Train_df.shape, Test_df.shape

((4619, 9), (1155, 9))

In [87]:
Train_df.reset_index(drop=True, inplace=True)
Test_df.reset_index(drop=True, inplace=True)

In [88]:
Train_dataset = Dataset.from_pandas(Train_df)
Test_dataset = Dataset.from_pandas(Test_df)

# **Test the Model with Zero Shot Inferencing**

**Using a pre-trained model from Hugging Face/sources - FLAN-T5-Base .**

**By testing with various models with the zero shot inferencing, we can see that the model struggles to extract the same subject line compared to the baseline subject, but it does pull out some important information from the email which indicates the models can be fine-tuned to the task at hand.**

# **Performing Full Fine-Tuning using FLAN-T5 BASE MODEL**

# **Pretrained('facebook/bart-base')**

In [4]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

In [10]:
# # Convert DataFrame to Dataset
# train_dataset = Dataset.from_pandas(train_df)
# test_dataset = Dataset.from_pandas(test_df)

# Initialize tokenizer and model
facebook_bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
facebook_bart_base_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [17]:
def print_number_of_trainable_model_parameters(model):
    """Prints the number of trainable parameters in a PyTorch model."""
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Number of trainable parameters: {trainable_params}")

print("For Model : facebook_bart_base_model \n",print_number_of_trainable_model_parameters(facebook_bart_base_model))

Number of trainable parameters: 139420416
For Model : facebook_bart_base_model 
 None


In [23]:
index = 1
subject = dataset_df['subject'][index]
email = dataset_df['body'][index]
email, subject

('Phillip & Keith  Attached is the first draw request, I will need some of these funds immediately.\nI think checks out of Bishops Corner, L.P. may be the easiest, or you can wire money to me and I can write the checks when needed.\nPlease let me know how you wish to handle this and I will proceed.\nAdditionally, it is getting close to closing.\nI will need to get contracts signed at this point as soon as possible.\nWe can handle this in two ways.\nI prefer that you elect me as a Vice - President of the General Partner and then I will sign all documents, or I can Federal Express contracts (this is more difficult).\nI have a funeral at 10:00 and I will be back about 1:00.\nI look forward to hearing from you.\nGreg     - Draw # 1 - 10.25.01.xls',
 'Bishops Corner')

In [28]:
# prompt = f"""
# Generate the subject line for the following email.

# Email:
# {email}

# Subject:
# """

inputs = facebook_bart_tokenizer(email, return_tensors='pt')

# Move input tensors to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {k: v.to(facebook_bart_base_model.device) for k, v in inputs.items()}

output_tokens = facebook_bart_base_model.generate(inputs['input_ids'], max_length=20, num_beams=4, early_stopping=True)
output = facebook_bart_tokenizer.decode(output_tokens[0], skip_special_tokens=True)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{email}')
print(dash_line)
print(f'BASELINE ORIGINAL HUMAN ANNOTED SUBJECT LINE :\n{subject}\n')
print(dash_line)
print(f'FB BART MODEL GENERATION - ZERO SHOT SUBJECT LINE:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Phillip & Keith  Attached is the first draw request, I will need some of these funds immediately.
I think checks out of Bishops Corner, L.P. may be the easiest, or you can wire money to me and I can write the checks when needed.
Please let me know how you wish to handle this and I will proceed.
Additionally, it is getting close to closing.
I will need to get contracts signed at this point as soon as possible.
We can handle this in two ways.
I prefer that you elect me as a Vice - President of the General Partner and then I will sign all documents, or I can Federal Express contracts (this is more difficult).
I have a funeral at 10:00 and I will be back about 1:00.
I look forward to hearing from you.
Greg     - Draw # 1 - 10.25.01.xls
---------------------------------------------------------------------------------------------------
BASELINE ORIGINAL HUMAN ANNOTED SUBJECT LINE

In [94]:
len(Train_df), len(Test_df)


(4619, 1155)

In [102]:
Train_df

,source,body,subject,ann0,ann1,ann2,body_wcount,subj_wcount,cleaned_emails
0,haedicke-m_inbox_205,This follows up on my voice-mail to you of jus...,In re F&J Representation,NaN,NaN,NaN,332,4,this follows up on my voicemail to you of just...
1,perlingiere-d_sent_530,"Twice in one day!\nI realize it is not Friday,...",Laclede Gas Company,NaN,NaN,NaN,45,3,twice in one day i realize it is not friday bu...
2,stclair-c_sent_2370,John: Thanks for your voice mail message.\nI w...,Misc,NaN,NaN,NaN,99,1,john thanks for your voice mail message. i wil...
3,skilling-j_inbox_384,"Ken and Jeff: Glen Darden, the President and ...",Quicksilver Resources,NaN,NaN,NaN,328,2,ken and jeff glen darden the president and ceo...
4,mann-k_sent_2331,I still plan to be out on Friday - guess I sho...,Calendar stuff,NaN,NaN,NaN,37,2,i still plan to be out on friday guess i shoul...
...,...,...,...,...,...,...,...,...,...
4614,campbell-l_inbox_613,"Dear Larry, It's the American Airlines Fare Sa...",Real Deals from Travelocity - Fare Sale Alert,NaN,NaN,NaN,901,8,dear larry its the american airlines fare sale...
4615,fossum-d_sent_1145,In anticipation of potential litigation involv...,TW Gas Sales: PRIVILEGED AND CONFIDENTIAL ATTO...,NaN,NaN,NaN,185,9,in anticipation of potential litigation involv...
4616,buy-r_inbox_1155,"I left a voice mail for David since he's out, ...",Frevert's Market Risk Report,NaN,NaN,NaN,186,4,i left a voice mail for david since hes out bu...
4617,cuilla-m_sent_16,I am interested in purchasing a Huglu shotgun ...,Huglushotguns,NaN,NaN,NaN,50,1,i am interested in purchasing a huglu shotgun ...


In [99]:
from torch.utils.data import Dataset

In [107]:
class EmailsDataset(Dataset):
    # Initialize the dataset with a tokenizer, data, and maximum token length
    def __init__(self, tokenizer, data, max_length=512):
        self.tokenizer = tokenizer  # Tokenizer for encoding text
        self.data  = data  # Data containing emials and subjects
        self.max_length = max_length # Maximum length of tokens

    # Return the number of items in the dataset
    def __len__(self):
        return len(self.data) # Change self.data to self.cleaned_emails

    # Retrieve an item from the dataset by index
    def __getitem__(self, idx):
        item = self.data.iloc[idx]  # Get the row at the specified index # Change self.data to self.cleaned_emails
        email = item['cleaned_emails'] # Extract emial from the row
        subject = item['subject']   # Extract summary from the row

        # Encode the dialogue as input data for the model
        source = self.tokenizer.encode_plus(
            email,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
            truncation=True
        )

        # Encode the summary as target data for the model
        target = self.tokenizer.encode_plus(
            subject,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
            truncation=True
        )

        # Return a dictionary containing input_ids, attention_mask, labels, and the original summary text
        return {
            'input_ids': source['input_ids'].flatten(),
            'attention_mask': source['attention_mask'].flatten(),
            'labels': target['input_ids'].flatten(),
            'summary': subject
        }

train_dataset = EmailsDataset(facebook_bart_tokenizer, Train_df)
test_dataset = EmailsDataset(facebook_bart_tokenizer, Test_df)
train_dataset, test_dataset

(<__main__.EmailsDataset at 0x78da3c0ee4d0>,
 <__main__.EmailsDataset at 0x78da3c0ee7a0>)

In [109]:
train_dataset

In [112]:
from transformers import TrainingArguments

# Define training arguments for the model
training_args = TrainingArguments(
    output_dir='./results',          # Directory to save model output and checkpoints
    num_train_epochs=2,              # Number of epochs to train the model
    per_device_train_batch_size=8,   # Batch size per device during training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Weight decay for regularization
    logging_dir='./logs',            # Directory to save logs
    fp16 = not torch.cuda.is_bf16_supported(),
    logging_steps=10,                # Log metrics every specified number of steps
    eval_strategy="epoch",     # Evaluation is done at the end of each epoch
    report_to='none'                 # Disables reporting to any online services (e.g., TensorBoard, WandB)
)

In [113]:
# Initializing the Trainer object
trainer = Trainer(
    model=facebook_bart_base_model,             # The model to be trained (e.g., our BART model)
    args=training_args,      # Training arguments specifying training parameters like learning rate, batch size, etc.
    train_dataset=train_dataset,  # The dataset to be used for training the model
    eval_dataset=test_dataset    # The dataset to be used for evaluating the model during training
)

# Starting the training process
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.053800,0.052637
2,0.045500,0.048044


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=1156, training_loss=1.33312296514177, metrics={'train_runtime': 1629.6263, 'train_samples_per_second': 5.669, 'train_steps_per_second': 0.709, 'total_flos': 2816372700610560.0, 'train_loss': 1.33312296514177, 'epoch': 2.0})

In [115]:
from transformers import GenerationConfig

In [116]:
# Save the final model checkpoint
facebook_bart_base_model.save_pretrained('./results/FB_Bart_model')
facebook_bart_tokenizer.save_pretrained('./results/FB_Bart_model')

# Define and save the generation configuration
generation_config = GenerationConfig(
    early_stopping=True,
    num_beams=4,
    no_repeat_ngram_size=3,
    forced_bos_token_id=0,
    forced_eos_token_id=2
)

generation_config.save_pretrained('./results/FB_Bart_model')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [117]:
model_directory = '/content/results/FB_Bart_model'

finetuned_facebook_bart_tokenizer = BartTokenizer.from_pretrained(model_directory)
finetuned_facebook_bart_base_model = BartForConditionalGeneration.from_pretrained(model_directory)

In [143]:

emails_for_scores_df = Test_df.sample(frac=0.02, random_state=42)
emails_for_scores_df.shape


(23, 9)

In [125]:
from datasets import load_metric
from torch.utils.data import DataLoader
import torch

In [144]:
emails_for_scores = EmailsDataset(facebook_bart_tokenizer, emails_for_scores_df)

In [146]:
from datasets import load_metric
from torch.utils.data import DataLoader

# Load the ROUGE metric for evaluation
rouge = load_metric('rouge')

def generate_summaries(model, tokenizer, dataset, batch_size=8):

    # Set model to evaluation mode
    model.eval()
    summaries = []    # List to store generated summaries
    references = []   # List to store actual summaries

    # Create a DataLoader for batch processing
    dataloader = DataLoader(dataset, batch_size=batch_size)

    # Disabled gradient calculations for efficiency
    with torch.no_grad():
        for batch in dataloader:
            # Move input data to the same device as the model
            input_ids = batch['input_ids'].to(model.device)
            attention_mask = batch['attention_mask'].to(model.device)

            # Generate summaries with the model
            outputs = model.generate(input_ids, attention_mask=attention_mask,generation_config=generation_config, max_length=20, num_beams=2)
            batch_summaries = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outputs]

            # Append generated and actual summaries to the respective lists
            summaries.extend(batch_summaries)
            references.extend(batch['summary'])

    return summaries, references

# Generate summaries for the validation dataset
generated_summaries, actual_summaries = generate_summaries(finetuned_facebook_bart_base_model, finetuned_facebook_bart_tokenizer, emails_for_scores, batch_size=8)

# Compute and print the ROUGE score for evaluation
rouge_score = rouge.compute(predictions=generated_summaries, references=actual_summaries)
rouge_score

Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


{'rouge1': AggregateScore(low=Score(precision=0.10286231884057973, recall=0.06715085638998687, fmeasure=0.07659937888198759), mid=Score(precision=0.2391304347826087, recall=0.15098814229249014, fmeasure=0.17443064182194618), high=Score(precision=0.40739130434782594, recall=0.25849802371541497, fmeasure=0.2887577639751553)),
 'rouge2': AggregateScore(low=Score(precision=0.043478260869565216, recall=0.010869565217391304, fmeasure=0.014492753623188408), mid=Score(precision=0.17391304347826086, recall=0.0753623188405797, fmeasure=0.09364548494983277), high=Score(precision=0.32608695652173914, recall=0.17274703557312251, fmeasure=0.1956521739130435)),
 'rougeL': AggregateScore(low=Score(precision=0.10576086956521744, recall=0.05991106719367592, fmeasure=0.0700776397515528), mid=Score(precision=0.2391304347826087, recall=0.1482213438735178, fmeasure=0.17080745341614906), high=Score(precision=0.39855072463768115, recall=0.26116271409749664, fmeasure=0.29038561076604563)),
 'rougeLsum': Aggreg

In [147]:
data = []
for rouge_type, scores in rouge_score.items():
    # Access scores directly using attributes
    row_low = {'type': rouge_type, 'level': 'low',
               'precision': scores.low.precision,
               'recall': scores.low.recall,
               'fmeasure': scores.low.fmeasure}
    row_mid = {'type': rouge_type, 'level': 'mid',
               'precision': scores.mid.precision,
               'recall': scores.mid.recall,
               'fmeasure': scores.mid.fmeasure}
    row_high = {'type': rouge_type, 'level': 'high',
               'precision': scores.high.precision,
               'recall': scores.high.recall,
               'fmeasure': scores.high.fmeasure}
    data.extend([row_low, row_mid, row_high])

scores_df = pd.DataFrame(data)

scores_df

,type,level,precision,recall,fmeasure
0,rouge1,low,0.102862,0.067151,0.076599
1,rouge1,mid,0.239130,0.150988,0.174431
2,rouge1,high,0.407391,0.258498,0.288758
3,rouge2,low,0.043478,0.010870,0.014493
4,rouge2,mid,0.173913,0.075362,0.093645
5,rouge2,high,0.326087,0.172747,0.195652
6,rougeL,low,0.105761,0.059911,0.070078
7,rougeL,mid,0.239130,0.148221,0.170807
8,rougeL,high,0.398551,0.261163,0.290386
9,rougeLsum,low,0.095652,0.057905,0.066656


In [153]:

original_subjects = []
fb_bart_subjects = []
finetuned_fb_bart_subjects = []


for index, row in emails_for_scores_df.iterrows():
    email = row['cleaned_emails']
    original_subject = row['subject']

    # Define the device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move both models to the same device
    facebook_bart_base_model.to(device)
    finetuned_facebook_bart_base_model.to(device)

    original_model_input_ids = facebook_bart_tokenizer(email, return_tensors="pt").input_ids.to(device)
    finetuned_model_input_ids = facebook_bart_tokenizer(email, return_tensors="pt").input_ids.to(device)


    original_model_outputs = facebook_bart_base_model.generate(input_ids=original_model_input_ids, num_beams=4,generation_config=generation_config)
    original_model_text_output = facebook_bart_tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    fb_bart_subjects.append(original_model_text_output)

    instruct_model_outputs = finetuned_facebook_bart_base_model.generate(input_ids=finetuned_model_input_ids, num_beams=4,generation_config=generation_config)
    instruct_model_text_output = facebook_bart_tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    finetuned_fb_bart_subjects.append(instruct_model_text_output)

    original_subjects.append(original_subject)

zipped_subjects = list(zip(original_subjects, fb_bart_subjects, finetuned_fb_bart_subjects))

FB_Bart_df = pd.DataFrame(zipped_subjects, columns = ['True_subjects', 'FB_Bart_subjects', 'Finetuned_FB_Bart_subjects'])
FB_Bart_df

,True_subjects,FB_Bart_subjects,Finetuned_FB_Bart_subjects
0,Enron Situation and Technology For All: A Note...,Technology for All,Technology for All
1,Organizational Announcement,Organizational Announcement,Organizational Announcement
2,Your approval is requested,MIGHOOD PRIVILEGED PRIVILEGED PRIVILEGED PRIVI...,MVP Request
3,Securities Loan Agreement,Enron Credit Inc.,Enron Credit Inc.
4,Request for Deal #,QQ6739.2,PG&E
5,Boxes,Boxes,Boxes
6,Volume Management Addition: Eugene Lee,Employee Announcement: Eugene Lee,Employee Announcement
7,Master Agreement,C constellation power source,C constellation power source
8,Charles' Christening,Meeting with charles,Meeting with charles
9,fantasy rules,Weekly pickups and drops,Bass


In [159]:

rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=fb_bart_subjects,
    references=original_subjects,
    use_aggregator=True,
    use_stemmer=True,
)

finetuned_model_results = rouge.compute(
    predictions=finetuned_fb_bart_subjects,
    references=original_subjects,
    use_aggregator=True,
    use_stemmer=True,
)

print('FB Bart Model:')
print(original_model_results)
print('\nFINETUNED FB Bart MODEL:')
print(finetuned_model_results)

FB Bart Model:
{'rouge1': 0.2591646276428885, 'rouge2': 0.10813823857302118, 'rougeL': 0.25725265073091164, 'rougeLsum': 0.25699385155906895}

FINETUNED FB Bart MODEL:
{'rouge1': 0.2881987577639752, 'rouge2': 0.12318840579710146, 'rougeL': 0.28799171842650106, 'rougeLsum': 0.2893374741200828}


In [156]:
import numpy as np

In [161]:
print("Absolute percentage improvement of FINETUNED MODEL over PRETRAINED")

improvement = (np.array(list(finetuned_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(finetuned_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of FINETUNED MODEL over PRETRAINED
rouge1: 2.90%
rouge2: 1.51%
rougeL: 3.07%
rougeLsum: 3.23%


In [3]:
# Step 2: Zip the folder
import shutil

zip_path = '/content/drive/MyDrive/models/facebook_bart_base.zip'

# Zip the folder
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', model_directory)

# Step 3: Download the zipped file
from google.colab import files

# Download the zip file
files.download(zip_path)

NameError: name 'model_directory' is not defined

In [165]:
from nltk.translate.meteor_score import meteor_score # Import meteor_score
import sacrebleu
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize # Import word_tokenize


In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# Initialize metric scorers
from rouge_score import rouge_scorer
rouge_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Evaluate the generated subject lines
def evaluate_metrics(true_subject, generated_subject):
    # ROUGE-L
    rougeL = rouge_scorer.score(true_subject, generated_subject)['rougeL'].fmeasure

    # METEOR
    meteor = meteor_score([word_tokenize(true_subject)], word_tokenize(generated_subject))

    # SACREBLEU
    sacre_bleu_score = sacrebleu.corpus_bleu([generated_subject], [[true_subject]]).score

    # BLEU
    bleu_score = sentence_bleu([word_tokenize(true_subject)], word_tokenize(generated_subject))


    return rougeL, meteor, sacre_bleu_score, bleu_score

# Apply metrics evaluation
FB_Bart_df['rougeL'], FB_Bart_df['meteor'], FB_Bart_df['sacrebleu'], FB_Bart_df['bleu'] = zip(*FB_Bart_df.apply(
    lambda row: evaluate_metrics(row['True_subjects'], row['Finetuned_FB_Bart_subjects']), axis=1))

# Calculate average scores
average_rougeL = FB_Bart_df['rougeL'].mean()
average_meteor = FB_Bart_df['meteor'].mean()
average_sacrebleu = FB_Bart_df['sacrebleu'].mean()
average_bleu = FB_Bart_df['bleu'].mean()


print(f'Average ROUGE-L: {average_rougeL:.4f}')
print(f'Average METEOR: {average_meteor:.4f}')
print(f'Average SACREBLEU: {average_sacrebleu:.4f}')
print(f'Average BLEU: {average_bleu:.4f}')

ModuleNotFoundError: No module named 'rouge_score'